In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# - **Author**: Alex Gao
# - **Date**: 2024-08-04
# - **Description**: This notebook is a part of Datathon 2024. The model used in this notebook is KAN model.

In [ ]:
# import pandas as pd
# import os

# def merge_and_filter_csv_files(directory):
#     # List to hold dataframes
#     dataframes = []
    
#     # Iterate over all files in the directory
#     for filename in os.listdir(directory):
#         if filename.endswith('.csv'):
#             file_path = os.path.join(directory, filename)
#             df = pd.read_csv(file_path)
#             dataframes.append(df)
    
#     # Concatenate all dataframes
#     merged_df = pd.concat(dataframes, ignore_index=True)
    
#     # Filter rows based on "Description" column containing specific strings
#     descriptions_to_keep = [
#         "All industry", 
#         "Food",
#         "GDP"
#     ]
    
#     # Filter rows
#     filtered_df = merged_df[merged_df["Description"].str.contains('|'.join(descriptions_to_keep)) | (merged_df.index == 0)]
    
#     # Sort the DataFrame according to "GeoName" with "United States" on top
#     filtered_df["GeoName"] = filtered_df["GeoName"].fillna("")
#     sorted_df = filtered_df.sort_values(by=["GeoName", "TableName"])
#     sorted_df = sorted_df.sort_values(by=["GeoName"], key=lambda x: x.str.contains("United States"), ascending=False)
    
#     return sorted_df

# # Directory where the CSV files are located
# directory = './data/SAGDP'

# # Merge, filter, and sort the DataFrame
# result_df = merge_and_filter_csv_files(directory)

# result_df.to_csv('SAGDP_summary.csv', index=False)

In [ ]:
# How do changes in employment rates correlate with processed food consumption?

# consumed = (percentage of processed meat) * (1 - percent waste) * (produced - stored)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
meat_stats_cold_storage = pd.read_csv("Datathon Data/Meat_Stats_Cold_Storage.csv")

meat_stats_meat_production = pd.read_csv("Datathon Data/Meat_Stats_Meat_Production.csv")
egg_rows = meat_stats_cold_storage['Animal'] == 'Frozen Eggs'
meat_stats_cold_storage_clean = meat_stats_cold_storage[~egg_rows]
incomplete_date_cold_storage = meat_stats_cold_storage_clean[meat_stats_cold_storage_clean['Weight'].isna()]

cold_storage_dates_to_remove = incomplete_date_cold_storage['Date'].unique()

meat_stats_cold_storage_clean = meat_stats_cold_storage_clean[~meat_stats_cold_storage_clean['Date'].isin(cold_storage_dates_to_remove)]

meat_stats_cold_storage_clean = meat_stats_cold_storage_clean.drop(columns=['Year', 'Month'])
meat_stats_cold_storage_clean = meat_stats_cold_storage_clean.groupby('Date')['Weight'].sum()
incomplete_date_meat_production = meat_stats_meat_production[meat_stats_meat_production['Production'].isna()]

meat_production_dates_to_remove = incomplete_date_meat_production['Date'].unique()

meat_stats_meat_production_clean = meat_stats_meat_production[~meat_stats_meat_production['Date'].isin(meat_production_dates_to_remove)]

meat_stats_meat_production_clean = meat_stats_meat_production_clean.drop(columns=['Year', 'Month'])
meat_stats_meat_production_clean['Production'] = meat_stats_meat_production_clean['Production'].str.replace(',', '').astype(float)
meat_stats_meat_production_clean = meat_stats_meat_production_clean.groupby('Date')['Production'].sum()
df_cold_storage = meat_stats_cold_storage_clean.reset_index()
df_cold_storage.columns = ['Date', 'Weight']

df_meat_production = meat_stats_meat_production_clean.reset_index()
df_meat_production.columns = ['Date', 'Production']

cold_storage_dates = set(df_cold_storage['Date'])
meat_production_dates = set(df_meat_production['Date'])
common_dates = cold_storage_dates.intersection(meat_production_dates)

filtered_df_cold_storage = df_cold_storage[df_cold_storage['Date'].isin(common_dates)]
filtered_df_meat_production = df_meat_production[df_meat_production['Date'].isin(common_dates)]
# We will assume there is 35% loss for the food

# So, consumed = (1 - 0.35) * (produced - stored)

# or consumed = 0.65*(produced - stored)
merged_df = pd.merge(filtered_df_cold_storage, filtered_df_meat_production, on='Date')

merged_df['Consumption'] = (merged_df['Production'] - merged_df['Weight'])*0.65

consumption_df = merged_df[['Date', 'Consumption']]


In [ ]:
consumption_df['Date'] = pd.to_datetime(consumption_df['Date'], format='%b-%Y')

# Format the 'Date' column to "Y-M"
consumption_df['Date'] = consumption_df['Date'].dt.strftime('%Y-%m')

In [ ]:
commodities_data = pd.read_csv('./data/all_commodities.csv')
stocks_etfs_data = pd.read_csv('./data/all_stock_and_etfs.csv')
exchange_rate_data = pd.read_csv('./data/ExchangeRate.csv')
cold_storage_data = pd.read_csv('./data/Meat_Stats_Cold_Storage.csv')
meat_production_data = pd.read_csv('./data/Meat_Stats_Meat_Production.csv')
stock_descriptions_data = pd.read_csv('./data/stock_descriptions.csv')

In [ ]:
commodities_data = commodities_data.drop(columns=['Unit'])

# Pivot the DataFrame to have commodities as columns
commodities_data_pivot = commodities_data.pivot(index='Date-Time', columns='Commodity', values='Value')

# Flatten the columns (if necessary, depends on pivot result)
commodities_data_pivot.columns = [f"{col}_Value" for col in commodities_data_pivot.columns]

# Reset the index to make 'Date-Time' a column again
commodities_data_pivot = commodities_data_pivot.reset_index()

commodities_data_pivot['Date-Time'] = pd.to_datetime(commodities_data_pivot['Date-Time'], format='%Y-%m-%d')

commodities_data_pivot['Date-Time'] = commodities_data_pivot['Date-Time'].dt.strftime('%Y-%m')

In [ ]:
stocks_etfs_data['Date-Time'] = pd.to_datetime(stocks_etfs_data['Date-Time'])
stocks_etfs_data['Date-Time'] = stocks_etfs_data['Date-Time'].dt.strftime('%Y-%m')
grouped_stock = stocks_etfs_data.groupby(['Date-Time', 'Ticker_Symbol']).mean().reset_index()

grouped_stock_flat = grouped_stock.pivot_table(
    index='Date-Time', 
    columns='Ticker_Symbol', 
    values=['Open', 'High', 'Low', 'Close', 'Volume'], 
    aggfunc='mean'
)
grouped_stock_flat.columns = [f'{col[1]}_{col[0]}' for col in grouped_stock_flat.columns]


In [ ]:
## and only take data from 1999-11 to 2024-02

# commodities_data_pivot
# grouped_stock_flat
# consumption_df

## these are the dataframes that we will merge with, basically merge the others into grouped_stock_flat

merged_df = pd.merge(grouped_stock_flat, commodities_data_pivot, on='Date-Time', how='outer')

merged_df = pd.merge(merged_df, consumption_df, left_on='Date-Time', right_on='Date', how='outer')

merged_df = merged_df[(merged_df['Date-Time'] >= '1999-11') & (merged_df['Date-Time'] <= '2024-02')]
desired_substrings = ['DIA', 'ONEQ', 'VOO', 'SPY']

# Step 2: Filter the columns that contain any of the desired substrings
selected_columns = [col for col in merged_df.columns if any(sub in col for sub in desired_substrings)]
## drop column "Date"
merged_df.drop(columns=['Date'], inplace=True)

In [ ]:
## normalize it, but merged_df first column is Date-Time, so we will normalize from the second column
## but stil keep the Date-Time column


mean = merged_df.iloc[:, 1:].mean()
std = merged_df.iloc[:, 1:].std()

## how to normalize merged_df?

merged_df.iloc[:, 1:] = (merged_df.iloc[:, 1:] - mean) / std
merged_df.fillna(0, inplace=True)

In [ ]:
## x_columns = columns except selected_columns

x_columns = [col for col in merged_df.columns if col not in selected_columns]

## discard "Date-Time" column

x_columns = x_columns[1:]


merged_df ['DIA_mean'] = merged_df[['DIA_Open', 'DIA_Close']].mean(axis=1)
merged_df ['ONEQ_mean'] = merged_df[['ONEQ_Open', 'ONEQ_Close']].mean(axis=1)
merged_df ['SPY_mean'] = merged_df[['SPY_Open', 'SPY_Close']].mean(axis=1)
merged_df ['VOO_mean'] = merged_df[['VOO_Open', 'VOO_Close']].mean(axis=1)

target_columns = [
    'DIA_mean','SPY_mean', 'VOO_mean'
]

# df_x = merged_df[x_columns]
# df_y = merged_df[target_columns]
# df_y = df_y.iloc[1:]
# df_x = df_x.iloc[:-1]

df_before2022 = merged_df[merged_df['Date-Time'] < '2022-01']

df_after2022 = merged_df[merged_df['Date-Time'] >= '2022-01']

df_x_train = df_before2022[x_columns]
df_y_train = df_before2022[target_columns]
df_x_val = df_after2022[x_columns]
df_y_val = df_after2022[target_columns]


In [ ]:
## drop the last column of df_x_train and df_x_val
## drop the first column of df_y_train and df_y_val

df_x_train = df_x_train.iloc[:-1]
df_x_val = df_x_val.iloc[:-1]
df_y_train = df_y_train.iloc[1:]
df_y_val = df_y_val.iloc[1:]

In [ ]:
## finally fill na with 0 again

df_x_train.fillna(0, inplace=True)
df_x_val.fillna(0, inplace=True)
df_y_train.fillna(0, inplace=True)
df_y_val.fillna(0, inplace=True)

x_tensor_train = torch.tensor(df_x_train.values, dtype=torch.float32)
y_tensor_train = torch.tensor(df_y_train.values, dtype=torch.float32)
x_tensor_val = torch.tensor(df_x_val.values, dtype=torch.float32)
y_tensor_val = torch.tensor(df_y_val.values, dtype=torch.float32)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

import torch.nn.functional as F

class SplineLinear(nn.Linear):
    def __init__(self, in_features: int, out_features: int, init_scale: float = 0.1, **kw) -> None:
        self.init_scale = init_scale
        super().__init__(in_features, out_features, bias=False, **kw)

    def reset_parameters(self) -> None:
        nn.init.trunc_normal_(self.weight, mean=0, std=self.init_scale)

class RadialBasisFunction(nn.Module):
    def __init__(
        self,
        grid_min: float = -2.,
        grid_max: float = 2.,
        num_grids: int = 8,
        denominator: float = None,  # larger denominators lead to smoother basis
    ):
        super().__init__()
        grid = torch.linspace(grid_min, grid_max, num_grids)
        self.grid = torch.nn.Parameter(grid, requires_grad=False)
        self.denominator = denominator or (grid_max - grid_min) / (num_grids - 1)

    def forward(self, x):
        return torch.exp(-((x[..., None] - self.grid) / self.denominator) ** 2)

class FastKANLayer(nn.Module):
    def __init__(
        self,
        input_dim: int,
        output_dim: int,
        grid_min: float = -2.,
        grid_max: float = 2.,
        num_grids: int = 8,
        use_base_update: bool = True,
        base_activation = nn.SiLU(),
        spline_weight_init_scale: float = 0.1,
    ) -> None:
        super().__init__()
        self.layernorm = nn.LayerNorm(input_dim)
        self.rbf = RadialBasisFunction(grid_min, grid_max, num_grids)
        self.spline_linear = SplineLinear(input_dim * num_grids, output_dim, spline_weight_init_scale)
        self.use_base_update = use_base_update
        if use_base_update:
            self.base_activation = base_activation
            self.base_linear = nn.Linear(input_dim, output_dim)

    def forward(self, x, time_benchmark=False):
        if not time_benchmark:
            spline_basis = self.rbf(self.layernorm(x))
        else:
            spline_basis = self.rbf(x)
        ret = self.spline_linear(spline_basis.view(*spline_basis.shape[:-2], -1))
        if self.use_base_update:
            base = self.base_linear(self.base_activation(x))
            ret = ret + base
        return ret

class KAN(nn.Module):
    def __init__(
        self,
        layers_hidden: list,
        grid_min: float = -2.,
        grid_max: float = 2.,
        num_grids: int = 8,
        use_base_update: bool = True,
        base_activation = nn.SiLU(),
        spline_weight_init_scale: float = 0.1,
    ) -> None:
        super().__init__()
        self.layers = nn.ModuleList([
            FastKANLayer(
                in_dim, out_dim,
                grid_min=grid_min,
                grid_max=grid_max,
                num_grids=num_grids,
                use_base_update=use_base_update,
                base_activation=base_activation,
                spline_weight_init_scale=spline_weight_init_scale,
            ) for in_dim, out_dim in zip(layers_hidden[:-1], layers_hidden[1:])
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def evaluate(model, val_features, val_labels, batch_size=3, device='cuda'):
    model.eval()
    val_preds = []
    val_labels_list = []

    with torch.no_grad():
        num_samples = val_features.shape[0]
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_features = val_features[start_idx:end_idx].to(device)
            batch_labels = val_labels[start_idx:end_idx].to(device)

            outputs = model(batch_features)
            val_preds.append(outputs.detach().cpu().numpy().flatten())
            val_labels_list.append(batch_labels.detach().cpu().numpy().flatten())

    val_preds = np.concatenate(val_preds)
    val_labels_list = np.concatenate(val_labels_list)

    # Calculate IC for all data
    val_ic_all, _ = pearsonr(val_preds, val_labels_list)

    # Calculate IC for |y| > 5
    mask_5 = np.abs(val_labels_list) > 0.5
    val_ic_gt_5, _ = pearsonr(val_preds[mask_5], val_labels_list[mask_5])

    # Calculate IC for |y| > 9
    mask_9 = np.abs(val_labels_list) > 0.9
    val_ic_gt_9, _ = pearsonr(val_preds[mask_9], val_labels_list[mask_9])

    # Calculate IC for |y| > 12
    mask_12 = np.abs(val_labels_list) > 1.2
    val_ic_gt_12, _ = pearsonr(val_preds[mask_12], val_labels_list[mask_12])

    # Clean up memory
    del val_features
    del val_labels
    del outputs

    # Return ICs
    return {
        'val_ic_all': val_ic_all,
        'val_ic_gt_5': val_ic_gt_5,
        'val_ic_gt_9': val_ic_gt_9,
        'val_ic_gt_12': val_ic_gt_12
    }

def train_model(model, x_train, y_train, x_val, y_val, num_epochs=50, batch_size=3, learning_rate=1e-3, device='cuda'):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    ics = []
    losses = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for batch_features, batch_labels in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * batch_features.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Evaluate on validation set
        val_metrics = evaluate(model, x_val, y_val, batch_size, device)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, '
              f'Val IC All: {val_metrics["val_ic_all"]:.4f}, '
              f'Val IC > 5: {val_metrics["val_ic_gt_5"]:.4f}, '
              f'Val IC > 9: {val_metrics["val_ic_gt_9"]:.4f}, '
              f'Val IC > 12: {val_metrics["val_ic_gt_12"]:.4f}')
        ics.append(val_metrics['val_ic_all'])
        losses.append(epoch_loss)
    return ics, losses
        
        



In [ ]:
# Assuming x_train, y_train, x_val, y_val are already defined

ics = []
losses = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model1 = KAN([x_tensor_train.shape[1], 64, 32, 16, y_tensor_train.shape[1]],
             use_base_update=True, base_activation=F.gelu)
ics, losses =train_model(model1, x_tensor_train, y_tensor_train, x_tensor_val, y_tensor_val, num_epochs=100,
            batch_size=10, learning_rate=1e-3, device=device)


In [ ]:
## plot ics and losses

plt.plot(ics, label='IC')
plt.plot(losses, label='Loss')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Assuming model1 is your trained model
# and x_tensor_val, y_tensor_val are your validation data

# Make predictions on validation set
y_pred = model1(x_tensor_val).detach().cpu().numpy()
y_true = y_tensor_val.detach().cpu().numpy()

# Plot predicted vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(y_true, y_pred, alpha=0.5)
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs Actual Values')
plt.show()

# Calculate and print performance metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)
print(f"RMSE: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Plot feature importance
feature_importance = np.abs(model1.layers[0].spline_linear.weight.detach().cpu().numpy()).mean(axis=0)
plt.figure(figsize=(12, 6))
plt.bar(range(len(feature_importance)), feature_importance)
plt.xlabel('Feature Index')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.show()

# Correlation heatmap of key variables
key_vars = ['DIA_mean', 'SPY_mean', 'VOO_mean', 'Consumption', 'Corn_Value']
corr_matrix = merged_df[key_vars].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Key Variables')
plt.show()

# Time series plot of actual vs predicted values for one ETF
etf_name = 'DIA_mean'
plt.figure(figsize=(12, 6))
plt.plot(merged_df['Date-Time'][-len(y_true):], y_true[:, 0], label='Actual')
plt.plot(merged_df['Date-Time'][-len(y_pred):], y_pred[:, 0], label='Predicted')
plt.xlabel('Date')
plt.ylabel(etf_name)
plt.title(f'Actual vs Predicted Values for {etf_name}')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()